In [1]:
#https://panel.holoviz.org/reference/panes/Vega.html
import warnings
warnings.filterwarnings('ignore')

import pandas as pd 
import panel as pn
pn.extension('vega')

import altair as alt
from vega_datasets import data

AttributeError: 'NoneType' object has no attribute 'split'

In [2]:
vegalite = {
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
  "data": {"url": "https://raw.githubusercontent.com/vega/vega/master/docs/data/barley.json"},
  "mark": "bar",
  "encoding": {
    "x": {"aggregate": "sum", "field": "yield", "type": "quantitative"},
    "y": {"field": "variety", "type": "nominal"},
    "color": {"field": "site", "type": "nominal"}
  }
}

In [3]:
vegalite

{'$schema': 'https://vega.github.io/schema/vega-lite/v5.json',
 'data': {'url': 'https://raw.githubusercontent.com/vega/vega/master/docs/data/barley.json'},
 'mark': 'bar',
 'encoding': {'x': {'aggregate': 'sum',
   'field': 'yield',
   'type': 'quantitative'},
  'y': {'field': 'variety', 'type': 'nominal'},
  'color': {'field': 'site', 'type': 'nominal'}}}

In [4]:
vgl_pane = pn.pane.Vega(vegalite, height=240)
vgl_pane

Vega(dict, height=240, selection=Selection)

In [5]:
vegalite['mark'] = 'area'
vgl_pane.param.trigger('object')

In [6]:
vgl_pane.object = {
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
  "data": {
    "url": "https://raw.githubusercontent.com/vega/vega/master/docs/data/disasters.csv"
  },
  "width": 600,
  "height": 400,
  "transform": [
    {"filter": "datum.Entity !== 'All natural disasters'"}
  ],
  "mark": {
    "type": "circle",
    "opacity": 0.8,
    "stroke": "black",
    "strokeWidth": 1
  },
  "encoding": {
    "x": {
        "field": "Year",
        "type": "quantitative",
        "axis": {"labelAngle": 90},
        "scale": {"zero": False}
    },
    "y": {
        "field": "Entity",
        "type": "nominal",
        "axis": {"title": ""}
    },
    "size": {
      "field": "Deaths",
      "type": "quantitative",
      "legend": {"title": "Annual Global Deaths", "clipHeight": 30},
      "scale": {"range": [0, 5000]}
    },
    "color": {"field": "Entity", "type": "nominal", "legend": None}
  }
}

In [7]:
responsive_spec = dict(vgl_pane.object, width='container')

vgl_responsive_pane = pn.pane.Vega(responsive_spec)
vgl_responsive_pane

Vega(dict, selection=Selection)

In [8]:
cars = data.cars()

chart = alt.Chart(cars).mark_circle(size=60).encode(
    x='Horsepower',
    y='Miles_per_Gallon',
    color='Origin',
    tooltip=['Name', 'Origin', 'Horsepower', 'Miles_per_Gallon']
).interactive()

altair_pane = pn.panel(chart)
altair_pane

Vega(Chart, selection=Selection)

In [9]:
altair_pane.object = chart.mark_circle(size=100)

In [10]:
penguins_url = "https://raw.githubusercontent.com/vega/vega/master/docs/data/penguins.json"

chart1 = alt.Chart(penguins_url).mark_point().encode(
    x=alt.X('Beak Length (mm):Q', scale=alt.Scale(zero=False)),
    y=alt.Y('Beak Depth (mm):Q', scale=alt.Scale(zero=False)),
    color='Species:N'
).properties(
    height=300,
    width=300,
)

chart2 = alt.Chart(penguins_url).mark_bar().encode(
    x='count()',
    y=alt.Y('Beak Depth (mm):Q', bin=alt.Bin(maxbins=30)),
    color='Species:N'
).properties(
    height=300,
    width=100
)

pn.panel(chart1 | chart2)

Vega(HConcatChart, selection=Selection)

In [12]:
df = pd.read_json(penguins_url)

brush = alt.selection_interval(name='brush')  # selection of type "interval"

chart = alt.Chart(penguins_url).mark_point().encode(
    x=alt.X('Beak Length (mm):Q', scale=alt.Scale(zero=False)),
    y=alt.Y('Beak Depth (mm):Q', scale=alt.Scale(zero=False)),
    color=alt.condition(brush, 'Species:N', alt.value('lightgray'))
).properties(
    width=250,
    height=250
).add_params(
    brush
)

vega_pane = pn.pane.Vega(chart, debounce=10)

vega_pane

Vega(Chart, debounce=10, selection=Selection)

In [14]:
vega_pane.selection

Selection(brush=None, name='Selection00124')

In [15]:
chart.to_dict()['params']

[{'name': 'brush', 'select': {'type': 'interval'}}]

In [16]:
multi = alt.selection_point(name='multi')  # selection of type "multi"

multi_chart = alt.Chart(penguins_url).mark_point().encode(
    x=alt.X('Beak Length (mm):Q', scale=alt.Scale(zero=False)),
    y=alt.Y('Beak Depth (mm):Q', scale=alt.Scale(zero=False)),
    color=alt.condition(multi, 'Species:N', alt.value('lightgray'))
).properties(
    width=250,
    height=250
).add_params(
    multi
)

vega_multi = pn.pane.Vega(multi_chart, debounce=10)

vega_multi

Vega(Chart, debounce=10, selection=Selection)

In [17]:
vega_multi.selection

Selection(multi=[], name='Selection00129')

In [18]:
df.iloc[vega_multi.selection.multi]

,Species,Island,Beak Length (mm),Beak Depth (mm),Flipper Length (mm),Body Mass (g),Sex


In [23]:
selection = {'Beak Length (mm)': [51.824, 53.952], 'Beak Depth (mm)': [18.796, 18.904]}

In [20]:
def filtered_table(selection):
    if not selection:
        return '## No selection'
    query = ' & '.join(
        f'{crange[0]:.3f} <= `{col}` <= {crange[1]:.3f}'
        for col, crange in selection.items()
    )
    return pn.Column(
        f'Query: {query}',
        pn.pane.DataFrame(df.query(query), width=600, height=300)
    )

pn.Row(vega_pane, pn.bind(filtered_table, vega_pane.selection.param.brush))

Row
    [0] Vega(Chart, debounce=10, selection=Selection)
    [1] ParamFunction(function, _pane=Markdown)

In [21]:
def bar_counts(selection):
    if not selection:
        return '## No selection'
    range_predicate = {
        'and': [{
            'field': key,
            'range': [selection[key][0], selection[key][1]]
        } for key in selection]
    }
    return alt.Chart(penguins_url, width=220).mark_bar().encode(
        x='count()',
        y='Species:N',
        color=alt.Color('Species:N', legend=None)
    ).transform_filter(
        range_predicate
    )

pn.Column(vega_pane, pn.bind(bar_counts, vega_pane.selection.param.brush))

Column
    [0] Vega(Chart, debounce=10, selection=Selection)
    [1] ParamFunction(function, _pane=Markdown)

In [24]:
query = ' & '.join([f'`{col}` in {values}' for col, values in selection.items()])

In [25]:
chart = alt.Chart(df).mark_tick().encode(
    x=alt.X('Beak Length (mm):Q', scale=alt.Scale(zero=False)),
    y='Species:N',
    color=alt.condition(brush, 'Species:N', alt.value('lightgray'))
).add_params(
    brush
)

def filtered_table(selection):
    if not selection:
        return '## No selection'
    query = ' & '.join(
        f'{values[0]} <= `{col}` <= {values[1]}'
        if pd.api.types.is_numeric_dtype(df[col])
        else f'`{col}` in {values}' 
        for col, values in selection.items()
    )
    return pn.Column(
        f'Query: {query}',
        pn.pane.DataFrame(df.query(query), width=600, height=300)
    )


vega_pane = pn.pane.Vega(chart, debounce=10)
pn.Row(vega_pane, pn.bind(filtered_table, vega_pane.selection.param.brush))

Row
    [0] Vega(Chart, debounce=10, selection=Selection)
    [1] ParamFunction(function, _pane=Markdown)

In [26]:
temps = data.seattle_temps()[:300]

brush = alt.selection_interval(name='brush')

chart = alt.Chart(temps).mark_circle().encode(
    x='date:T',
    y=alt.Y('temp:Q', scale={'zero': False}),
    color=alt.condition(brush, alt.value('coral'), alt.value('lightgray'))
).properties(
    width=500
).add_params(
    brush
)

def filtered_table(selection):
    if not selection:
        return '## No selection'
    query = ' & '.join(
        f'"{pd.to_datetime(values[0], unit="ms")}" <= `{col}` <= "{pd.to_datetime(values[1], unit="ms")}"'
        if pd.api.types.is_datetime64_any_dtype(temps[col]) else f'{values[0]} <= `{col}` <= {values[1]}'
        for col, values in selection.items()
    )
    return pn.Column(
        f'Query: {query}',
        pn.pane.DataFrame(temps.query(query), width=600, height=300)
    )


vega_pane = pn.pane.Vega(chart, debounce=10)
pn.Row(vega_pane, pn.bind(filtered_table, vega_pane.selection.param.brush))

Row
    [0] Vega(Chart, debounce=10, selection=Selection)
    [1] ParamFunction(function, _pane=Markdown)

In [27]:
pn.Row(vgl_responsive_pane.controls(jslink=True), vgl_responsive_pane, sizing_mode="stretch_width")

Row(sizing_mode='stretch_width')
    [0] Tabs
        [0] WidgetBox(name='Controls', width=300)
            [0] StaticText(value='<b>Controls</b>')
            [1] Checkbox(name='Show actions')
            [2] Select(name='Theme', options=OrderedDict([('excel', ...]), value='excel')
        [1] WidgetBox(name='Layout', width=300)
            [0] StaticText(value='<b>Layout</b>')
            [1] TextInput(disabled=True, name='Name', value='Vega00107')
            [2] LiteralInput(name='Align', serializer='json', value='start')
            [3] LiteralInput(name='Aspect ratio', serializer='json')
            [4] LiteralInput(name='Background', serializer='json')
            [5] ListInput(name='Css classes', serializer='json', type=<class 'list'>)
            [6] IntInput(name='Width', start=0)
            [7] IntInput(name='Height', start=0)
            [8] IntInput(name='Min width', start=0)
            [9] IntInput(name='Min height', start=0)
            [10] IntInput(name='Max width', start=0)
            [11] IntInput(name='Max height', start=0)
            [12] LiteralInput(name='Margin', serializer='json', value=(5, 5, 30, 5))
            [13] Select(name='Width policy', options=OrderedDict([('auto', ...]), value='auto')
            [14] Select(name='Height policy', options=OrderedDict([('auto', ...]), value='auto')
            [15] Select(name='Sizing mode', options=OrderedDict([('fixed', ...]))
            [16] Checkbox(name='Visible', value=True)
    [1] Vega(dict, selection=Selection)